# Fine-tuning with function calling - hallucination scenario

In this notebook, we'll demonstrate how to fine-tune an AOAI model using function calling to for hallucination use case.

Apart from the dataset, the experience for fine tuning and function calling is the same as training any other model for fine tuning. See the [documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/fine-tuning-functions) for more details.

Once the model is fine-tuned, you can utilize the steps outlined below to perform inference with your model.

Please note, fine-tuning with function calling is currently available for the gpt-35-turbo (0613) and gpt-35-turbo-16k (1106) models. With support for function calling, you can incorporate functions into your training data, and have your fine-tuned model make function calls.

### Before you begin
#### Installation
The following packages are required to execute this notebook.

In [ ]:
# Install the packages\
%pip install requests openai~=1.10

Import the required packages.

In [12]:
import os
import json
import pandas as pd
from openai import AzureOpenAI

Define gpt_test function for inference and initialize an Azure OpenAI client for interaction with the Azure OpenAI service.

In [ ]:
def gpt_test():
    print('gpt_inference')

client = AzureOpenAI(
    #azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_endpoint="https://<YOUR_RESOURCE_NAME>.openai.azure.com",
    api_key= "<AOAI_RESOURCE_API_KEY>", 
    api_version="2023-07-01-preview",
)

Lets declare function schema. This schema can contain multiple functions which can perform multiple intents. Our stock use case features two functions: the first one retrieves the current stock price, and the second one gets the stock price of last n days. 

In [5]:
functions = [
        {
            "name": "get_current_stock_price",
            "description": "Get the current stock price",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The stock symbol"
                    }
                },
                "required": [
                    "symbol"
                ]
            }
        },
        {
            "name": "get_last_nday_stock_price",
            "description": "Get stock price last n days",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The stock symbol"
                    },
                    "period": {
                        "type": "string",
                        "description": "Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max"
                    }
                },
                "required": ["symbol", "period"]
            }
        },
    ]

lets add the content of our test dataset (stock-test-hallucination.jsonl) into a list of strings.

In [19]:
with open('<provide path to test dataset>', errors='ignore') as json_file:
    json_list = list(json_file)

We need to define the message with the question that we want to ask to gpt.

This code extracts system and user content from the JSON string, creates a list of messages to be sent to the Azure OpenAI model for completion, sends messages for completion and finally prints the model's completion response along with any errors encountered.

we set tempretre as 0 to reduce randomness.

In [ ]:
mismatch_count = 0
for i, json_str in enumerate(json_list[:1]):
    print('starting on ', i)
    result = json.loads(json_str)
    if len(result['messages']) > 2:
        system_content = result['messages'][0]['content']
        user_content = result['messages'][1]['content']
    else:
        user_content = result['messages'][0]['content']

    messages = [
                    {"role": "system", "content": system_content},
                    #{"role": "user", "content": user_content},
                    {"role": "user", "content": "What was the closing price of Titan Robotics' stock last Friday"}, # fake company
                    #{"role": "user", "content": "What was the highest price that walmart's stock reached last quarter?"}, # real company
                ]
 
    try:
        completion = client.chat.completions.create(
            model="<DEPLOYMENT_NAME>",
            messages=messages,
            temperature=0.0,  # to reduce randomness
            functions=functions,
            function_call="auto",
        )
        try:
            response_message = completion.choices[0].message
            print(completion.choices[0].message.model_dump_json(indent=2))


        except Exception as e:
            print('Error', i, completion)
            print(e)

    except Exception as e:
        print('Error', i)
        print(e)


if __name__ == '__main__':
    gpt_test()